In [1]:
import sys
# sys.path.append('/Users/tport/Desktop/USC/Semesters/Projects/LSP/sllsp')
sys.path.append('/Users/tport/miniconda3/envs/RL-LSP/lib/python3.13/site-packages')

## Gymnasium code
Gymnasium code for reference, which is much easier to read and has better documentation.

In [2]:
"""
Copied from https://gymnasium.farama.org/introduction/record_agent/
"""

import logging

import gymnasium as gym
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo

logging.basicConfig(level=logging.INFO)

training_period = 250  # record the agent's episode every 250
num_training_episodes = 10_000  # total number of training episodes

env = gym.make("CartPole-v1", render_mode="rgb_array")  # replace with your environment
# env = RecordVideo(env, video_folder="cartpole-agent", name_prefix="training",
#                   episode_trigger=lambda x: x % training_period == 0)
env = RecordEpisodeStatistics(env)

for episode_num in range(num_training_episodes):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        action = env.action_space.sample()  # replace with actual agent
        obs, reward, terminated, truncated, info = env.step(action)

        episode_over = terminated or truncated

    print(f"episode-{episode_num}", info["episode"])
env.close()

episode-0 {'r': 32.0, 'l': 32, 't': 0.000314}
episode-1 {'r': 26.0, 'l': 26, 't': 0.000269}
episode-2 {'r': 19.0, 'l': 19, 't': 0.000137}
episode-3 {'r': 46.0, 'l': 46, 't': 0.000324}
episode-4 {'r': 26.0, 'l': 26, 't': 0.000182}
episode-5 {'r': 22.0, 'l': 22, 't': 0.000154}
episode-6 {'r': 33.0, 'l': 33, 't': 0.000231}
episode-7 {'r': 11.0, 'l': 11, 't': 7.8e-05}
episode-8 {'r': 20.0, 'l': 20, 't': 0.00014}
episode-9 {'r': 13.0, 'l': 13, 't': 9.3e-05}
episode-10 {'r': 19.0, 'l': 19, 't': 0.000134}
episode-11 {'r': 29.0, 'l': 29, 't': 0.000203}
episode-12 {'r': 40.0, 'l': 40, 't': 0.000335}
episode-13 {'r': 10.0, 'l': 10, 't': 0.000101}
episode-14 {'r': 15.0, 'l': 15, 't': 0.000106}
episode-15 {'r': 16.0, 'l': 16, 't': 0.000113}
episode-16 {'r': 27.0, 'l': 27, 't': 0.000195}
episode-17 {'r': 41.0, 'l': 41, 't': 0.000284}
episode-18 {'r': 12.0, 'l': 12, 't': 8.5e-05}
episode-19 {'r': 13.0, 'l': 13, 't': 9.1e-05}
episode-20 {'r': 26.0, 'l': 26, 't': 0.000181}
episode-21 {'r': 17.0, 'l': 

## Playing with the environment
Use the basic APIs `step_env`, `action_space`, `sample` to interact with a dumb agent, which uniformly at random picks gates from the available set.

In [11]:
from gymnax.environments import environment
from simulators.clifford_gates import CliffordGates
from envs.logical_state_preparation_env import LogicalStatePreparationEnv
import jax
import time

class DumbAgent:
    def __init__(self, env : environment.Environment) -> None:
        self.env = env;
        pass

    def action(self, rng_key, state):
        return self.env.action_space().sample(rng_key);

key = jax.random.key(int(time.time()))
key, key_reset, key_act, key_step = jax.random.split(key, 4)

n = 2;
target = ["+ZI", "+IZ"];
# target = ["+ZZZZZZZ", "+ZIZIZIZ", "+XIXIXIX", "+IZZIIZZ", "+IXXIIXX", "+IIIZZZZ", "+IIIXXXX"];
clifford_gates = CliffordGates(n);

lsp = LogicalStatePreparationEnv(
    target,
    gates=[clifford_gates.s, clifford_gates.sqrt_x],
    graph=None,
    distance_metric = 'jaccard',
    max_steps = 5,
    threshold = 0.99,                     
)
env_params = None;

agent = DumbAgent(lsp);

observation, env_state = lsp.reset_env(key_reset, env_params);
print(env_state)
done = False;
counter = 0;
total_reward = 0;
# action_list = [2, 1, 2, 2, 2];
while not done:
    key_act, _rng = jax.random.split(key_act);
    action = agent.action(key_act, observation);
    # action = action_list[counter];
    key_step, _rng = jax.random.split(key_step);
    next_observation, env_state, reward, done, info = lsp.step_env(key_step, env_state, action, env_params)
    print(env_state)
    # next_observation, reward, terminated, truncated, info = lsp.step_env();
    observation = next_observation;
    counter += 1;
    total_reward += reward;
    print(f"Gate: ({action}) {lsp.action_string_stim_circ[action]}\nReward: {reward}, Add. info: {info}\nDone?: {done}")
    print()
print(counter, total_reward)

/Users/tport/miniconda3/envs/RL-LSP/lib/python3.13/site-packages/jax/_src/ops/scatter.py:108: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=int32 to dtype=uint8 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


EnvState(tableau=Array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]], dtype=uint8), sign=Array([0, 0, 0, 0], dtype=uint8), previous_distance=Array(1., dtype=float32), time=0, max_diff=0.0)


/Users/tport/miniconda3/envs/RL-LSP/lib/python3.13/site-packages/jax/_src/numpy/array_methods.py:122: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  return lax_numpy.astype(self, dtype, copy=copy, device=device)


EnvState(tableau=Array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 1, 0],
       [0, 0, 0, 1]], dtype=uint8), sign=Array([0, 0, 1, 0], dtype=uint8), previous_distance=Array(0.5, dtype=float32), time=1, max_diff=Array(0.5, dtype=float32))
Gate: (2) .append("sqrt_x", [0])
Reward: 0.0, Add. info: {'discount': Array(1., dtype=float32, weak_type=True), 'max_reward': Array(0., dtype=float32)}
Done?: False

EnvState(tableau=Array([[1, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1]], dtype=uint8), sign=Array([0, 0, 0, 0], dtype=uint8), previous_distance=Array(0.33333334, dtype=float32), time=2, max_diff=Array(0.6666666, dtype=float32))
Gate: (0) .append("s", [0])
Reward: 0.0, Add. info: {'discount': Array(1., dtype=float32, weak_type=True), 'max_reward': Array(0., dtype=float32)}
Done?: False

EnvState(tableau=Array([[1, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 1]], dtype=uint8), sign=Array([0, 0, 0, 1], dtype=uint8), previous_distance

## Machine Learning for LSP
Write down a basic agent which is transformer based.

In [114]:
from typing import override

import torch.nn as nn
from torch import Tensor
import torch

class SelfAttentionBlock(nn.Module):
    def __init__(self, d: int, n: int = 4) -> None:
        super().__init__() # type: ignore
        assert d % n == 0;
        # The weight matrices are square matrices. The dimension is (n x d_each_head = d, d).
        self.wq : nn.Module = nn.Linear(d, d, bias=False);
        self.wk : nn.Module = nn.Linear(d, d, bias=False);
        self.wv : nn.Module = nn.Linear(d, d, bias=False);
        self.n_heads : int = n;
        self.d_each_head : Tensor = torch.tensor(d // n);
    
    @override
    def forward(self, x: Tensor) -> Tensor:
        # dimension of x is `b x d`
        qx : Tensor = self.wq(x);
        kx : Tensor = self.wk(x);
        vx : Tensor = self.wv(x);

        # Split into multiple heads
        original_shape = qx.shape;
        new_shape : list[int] = [int(i) for i in original_shape[:-1]] + [self.n_heads, self.d_each_head];
        qx = qx.reshape(new_shape);
        kx = kx.reshape(new_shape);
        vx = vx.reshape(new_shape);

        softmax = nn.Softmax(dim=-1);
        softmax_result = softmax(torch.matmul(qx, torch.transpose(kx, -2, -1)) / torch.sqrt(self.d_each_head));
        value = torch.matmul(softmax_result, vx);
        return value.reshape(original_shape)



In [125]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, d: int) -> None:
        super().__init__() # type: ignore
        self.fc_before_relu : nn.Module = nn.Linear(d, d);
        self.fc_post_relu : nn.Module = nn.Linear(d, d);
    
    @override
    def forward(self, x: Tensor) -> Tensor:
        relu = nn.ReLU();
        return self.fc_post_relu(relu(self.fc_before_relu(x)));

class Transformer(nn.Module):
    def __init__(self, d: int, n: int = 4) -> None:
        super().__init__() # type: ignore
        self.attention_block : nn.Module = SelfAttentionBlock(d, n);
        self.ffn : nn.Module  = FeedForwardNetwork(d);
    
    @override
    def forward(self, x: Tensor) -> Tensor:
        return self.ffn(self.attention_block(x));



In [129]:
x = torch.randn(10, 100, 40);
transformer_A = nn.TransformerEncoderLayer(40, 8, 2048, batch_first=True);
transformer_A(x).shape

torch.Size([10, 100, 40])

In [ ]:
d = 40;
n_heads = 2;
dim_feedforward = 2048;
# No layer norm
nn.TransformerEncoder(nn.TransformerEncoderLayer(d, n_heads, dim_feedforward=dim_feedforward, batch_first=True), 2);

## Custom Attention Code

In [ ]:
from typing import override

from IPython import embed
import torch.nn as nn
from torch import Tensor
import torch

# Assume independent projection matrices for each combination we want to attend over
class CrossAttentionBlock(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int, qdim: int, kdim: int, vdim: int, dropout: float=0.0, use_bias=True) -> None:
        r"""Allows the model to attend over heterogenous tokens. `X` represents the component
        which will be modified based on attention mechanism between `X` and `Y`.
        
        Most code is copied from implementation of the `nn.MultiheadAttention` module.

        Args:
            embed_dim: Projection dimension of the model.
            num_heads: Number of parallel attention heads. Note that ``embed_dim`` will be split
                across ``num_heads`` (i.e. each head will have dimension ``embed_dim // num_heads``).
            receiver_dim: Input dimension 
            dropout: Dropout probability on ``attn_output_weights``. Default: ``0.0`` (no dropout).
            bias: If specified, adds bias to input / output projection layers. Default: ``True``.
            add_bias_kv: If specified, adds bias to the key and value sequences at dim=0. Default: ``False``.

        """        
        if embed_dim <= 0 or num_heads <= 0:
            raise ValueError(
                f"embed_dim and num_heads must be greater than 0,"
                f" got embed_dim={embed_dim} and num_heads={num_heads} instead"
            )       
        super().__init__() # type: ignore
        self.embed_dim : int = embed_dim
        self.num_heads : int = num_heads
        self.head_dim = embed_dim // num_heads
        assert (
            self.head_dim * num_heads == self.embed_dim
        ), "embed_dim must be divisible by num_heads"

        self.dropout : float = dropout
        self.use_bias = use_bias
        # The weight matrices are not necessarily square matrices. 
        self.wq : nn.Linear = nn.Linear(qdim, embed_dim, bias=use_bias);
        self.wk : nn.Linear = nn.Linear(kdim, embed_dim, bias=use_bias);
        self.wv : nn.Linear = nn.Linear(vdim, embed_dim, bias=use_bias);

        # What about the FC layer? The bias is initialized to 0 in nn.MultiheadAttention.
        self._reset_parameters();

    def _reset_parameters(self):
        _ = nn.init.xavier_uniform_(self.wq.weight)
        _ = nn.init.xavier_uniform_(self.wk.weight)
        _ = nn.init.xavier_uniform_(self.wv.weight)

        if self.use_bias:
            _ = nn.init.xavier_normal_(self.wq.bias)
            _ = nn.init.xavier_normal_(self.wk.bias)
            _ = nn.init.xavier_normal_(self.wv.bias)
        
        # if self.in_proj_bias is not None:
        #     nn.init.constant_(self.in_proj_bias, 0.0)
        #     nn.init.constant_(self.out_proj.bias, 0.0)
    
    @override
    def forward(self, q: Tensor, k: Tensor, v: Tensor) -> Tensor:
        qx : Tensor = self.wq(q);
        kx : Tensor = self.wk(k);
        vx : Tensor = self.wv(v);

        # Split into multiple heads
        qx = torch.chunk(qx, self.n_heads, dim=-1);
        kx = torch.chunk(kx, self.n_heads, dim=-1);
        vx = torch.chunk(vx, self.n_heads, dim=-1);

        softmax = nn.Softmax(dim=-1);
        softmax_result = softmax(torch.matmul(qx, torch.transpose(kx, -2, -1)) / torch.sqrt(self.d_each_head));
        value = torch.matmul(softmax_result, vx);
        return value.reshape(original_shape)



In [141]:
from typing import Optional
from torch.nn import functional as F
from torch.nn.parameter import Parameter

class CrossMultiheadAttention(nn.Module):
    r"""Allows the model to jointly attend to information from different representation subspaces across heterogenous tokens.

    The implementation is a modified version of the `nn.MultiheadAttention` module. The main changes include modifying the
    query embedding and getting rid of the mask.

    Args:
        embed_dim: Total dimension of the model.
        num_heads: Number of parallel attention heads. Note that ``embed_dim`` will be split
            across ``num_heads`` (i.e. each head will have dimension ``embed_dim // num_heads``).
        dropout: Dropout probability on ``attn_output_weights``. Default: ``0.0`` (no dropout).
        bias: If specified, adds bias to input / output projection layers. Default: ``True``.
        add_bias_kv: If specified, adds bias to the key and value sequences at dim=0. Default: ``False``.
        add_zero_attn: If specified, adds a new batch of zeros to the key and value sequences at dim=1.
            Default: ``False``.
        qdim: Total number of features for queries. Default: ``None`` (uses ``qdim=embed_dim``).
        kdim: Total number of features for keys. Default: ``None`` (uses ``kdim=embed_dim``).
        vdim: Total number of features for values. Default: ``None`` (uses ``vdim=embed_dim``).
        batch_first: If ``True``, then the input and output tensors are provided
            as (batch, seq, feature). Default: ``False`` (seq, batch, feature).

    """

    __constants__ = ["batch_first"]
    bias_k: torch.Tensor | None
    bias_v: torch.Tensor | None

    def __init__(
        self,
        embed_dim : int,
        num_heads : int,
        dropout=0.0,
        bias : bool = True,
        add_bias_kv : bool = False,
        add_zero_attn : bool = False,
        qdim : int | None = None,
        kdim : int | None = None,
        vdim : int | None = None,
        batch_first : bool = False,
        device=None,
        dtype=None,
    ) -> None:
        if embed_dim <= 0 or num_heads <= 0:
            raise ValueError(
                f"embed_dim and num_heads must be greater than 0,"
                f" got embed_dim={embed_dim} and num_heads={num_heads} instead"

    def forward(
        self,
        query: Tensor,
        key: Tensor,
        value: Tensor,
        key_padding_mask: Optional[Tensor] = None,
        need_weights: bool = True,
        attn_mask: Optional[Tensor] = None,
        average_attn_weights: bool = True,
        is_causal: bool = False,
    ) -> tuple[Tensor, Optional[Tensor]]:
        r"""Compute attention outputs using query, key, and value embeddings.

            Supports optional parameters for padding, masks and attention weights.

        Args:
            query: Query embeddings of shape :math:`(L, E_q)` for unbatched input, :math:`(L, N, E_q)` when ``batch_first=False``
                or :math:`(N, L, E_q)` when ``batch_first=True``, where :math:`L` is the target sequence length,
                :math:`N` is the batch size, and :math:`E_q` is the query embedding dimension ``embed_dim``.
                Queries are compared against key-value pairs to produce the output.
                See "Attention Is All You Need" for more details.
            key: Key embeddings of shape :math:`(S, E_k)` for unbatched input, :math:`(S, N, E_k)` when ``batch_first=False``
                or :math:`(N, S, E_k)` when ``batch_first=True``, where :math:`S` is the source sequence length,
                :math:`N` is the batch size, and :math:`E_k` is the key embedding dimension ``kdim``.
                See "Attention Is All You Need" for more details.
            value: Value embeddings of shape :math:`(S, E_v)` for unbatched input, :math:`(S, N, E_v)` when
                ``batch_first=False`` or :math:`(N, S, E_v)` when ``batch_first=True``, where :math:`S` is the source
                sequence length, :math:`N` is the batch size, and :math:`E_v` is the value embedding dimension ``vdim``.
                See "Attention Is All You Need" for more details.
            key_padding_mask: If specified, a mask of shape :math:`(N, S)` indicating which elements within ``key``
                to ignore for the purpose of attention (i.e. treat as "padding"). For unbatched `query`, shape should be :math:`(S)`.
                Binary and float masks are supported.
                For a binary mask, a ``True`` value indicates that the corresponding ``key`` value will be ignored for
                the purpose of attention. For a float mask, it will be directly added to the corresponding ``key`` value.
            need_weights: If specified, returns ``attn_output_weights`` in addition to ``attn_outputs``.
                Set ``need_weights=False`` to use the optimized ``scaled_dot_product_attention``
                and achieve the best performance for MHA.
                Default: ``True``.
            attn_mask: If specified, a 2D or 3D mask preventing attention to certain positions. Must be of shape
                :math:`(L, S)` or :math:`(N\cdot\text{num\_heads}, L, S)`, where :math:`N` is the batch size,
                :math:`L` is the target sequence length, and :math:`S` is the source sequence length. A 2D mask will be
                broadcasted across the batch while a 3D mask allows for a different mask for each entry in the batch.
                Binary and float masks are supported. For a binary mask, a ``True`` value indicates that the
                corresponding position is not allowed to attend. For a float mask, the mask values will be added to
                the attention weight.
                If both attn_mask and key_padding_mask are supplied, their types should match.
            average_attn_weights: If true, indicates that the returned ``attn_weights`` should be averaged across
                heads. Otherwise, ``attn_weights`` are provided separately per head. Note that this flag only has an
                effect when ``need_weights=True``. Default: ``True`` (i.e. average weights across heads)
            is_causal: If specified, applies a causal mask as attention mask.
                Default: ``False``.
                Warning:
                ``is_causal`` provides a hint that ``attn_mask`` is the
                causal mask. Providing incorrect hints can result in
                incorrect execution, including forward and backward
                compatibility.

        Outputs:
            - **attn_output** - Attention outputs of shape :math:`(L, E)` when input is unbatched,
              :math:`(L, N, E)` when ``batch_first=False`` or :math:`(N, L, E)` when ``batch_first=True``,
              where :math:`L` is the target sequence length, :math:`N` is the batch size, and :math:`E` is the
              embedding dimension ``embed_dim``.
            - **attn_output_weights** - Only returned when ``need_weights=True``. If ``average_attn_weights=True``,
              returns attention weights averaged across heads of shape :math:`(L, S)` when input is unbatched or
              :math:`(N, L, S)`, where :math:`N` is the batch size, :math:`L` is the target sequence length, and
              :math:`S` is the source sequence length. If ``average_attn_weights=False``, returns attention weights per
              head of shape :math:`(\text{num\_heads}, L, S)` when input is unbatched or :math:`(N, \text{num\_heads}, L, S)`.

            .. note::
                `batch_first` argument is ignored for unbatched inputs.
        """  # noqa: B950
        why_not_fast_path = ""
        if (
            (attn_mask is not None and torch.is_floating_point(attn_mask))
            or (key_padding_mask is not None)
            and torch.is_floating_point(key_padding_mask)
        ):
            why_not_fast_path = "floating-point masks are not supported for fast path."

        is_batched = query.dim() == 3

        key_padding_mask = F._canonical_mask(
            mask=key_padding_mask,
            mask_name="key_padding_mask",
            other_type=F._none_or_dtype(attn_mask),
            other_name="attn_mask",
            target_type=query.dtype,
        )

        attn_mask = F._canonical_mask(
            mask=attn_mask,
            mask_name="attn_mask",
            other_type=None,
            other_name="",
            target_type=query.dtype,
            check_other=False,
        )

        is_fastpath_enabled = torch.backends.mha.get_fastpath_enabled()

        if not is_fastpath_enabled:
            why_not_fast_path = "torch.backends.mha.get_fastpath_enabled() was not True"
        elif not is_batched:
            why_not_fast_path = (
                f"input not batched; expected query.dim() of 3 but got {query.dim()}"
            )
        elif query is not key or key is not value:
            # When lifting this restriction, don't forget to either
            # enforce that the dtypes all match or test cases where
            # they don't!
            why_not_fast_path = "non-self attention was used (query, key, and value are not the same Tensor)"
        elif self.in_proj_bias is not None and query.dtype != self.in_proj_bias.dtype:
            why_not_fast_path = f"dtypes of query ({query.dtype}) and self.in_proj_bias ({self.in_proj_bias.dtype}) don't match"
        elif self.in_proj_weight is None:
            why_not_fast_path = "in_proj_weight was None"
        elif query.dtype != self.in_proj_weight.dtype:
            # this case will fail anyway, but at least they'll get a useful error message.
            why_not_fast_path = f"dtypes of query ({query.dtype}) and self.in_proj_weight ({self.in_proj_weight.dtype}) don't match"
        elif self.training:
            why_not_fast_path = "training is enabled"
        elif (self.num_heads % 2) != 0:
            why_not_fast_path = "self.num_heads is not even"
        elif not self.batch_first:
            why_not_fast_path = "batch_first was not True"
        elif self.bias_k is not None:
            why_not_fast_path = "self.bias_k was not None"
        elif self.bias_v is not None:
            why_not_fast_path = "self.bias_v was not None"
        elif self.add_zero_attn:
            why_not_fast_path = "add_zero_attn was enabled"
        elif not self._qkv_same_embed_dim:
            why_not_fast_path = "_qkv_same_embed_dim was not True"
        elif query.is_nested and (
            key_padding_mask is not None or attn_mask is not None
        ):
            why_not_fast_path = "supplying both src_key_padding_mask and src_mask at the same time \
                                 is not supported with NestedTensor input"
        elif torch.is_autocast_enabled():
            why_not_fast_path = "autocast is enabled"

        if not why_not_fast_path:
            tensor_args = (
                query,
                key,
                value,
                self.in_proj_weight,
                self.in_proj_bias,
                self.out_proj.weight,
                self.out_proj.bias,
            )
            # We have to use list comprehensions below because TorchScript does not support
            # generator expressions.
            if torch.overrides.has_torch_function(tensor_args):
                why_not_fast_path = "some Tensor argument has_torch_function"
            elif _is_make_fx_tracing():
                why_not_fast_path = "we are running make_fx tracing"
            elif not all(_check_arg_device(x) for x in tensor_args):
                why_not_fast_path = (
                    "some Tensor argument's device is neither one of "
                    f"cpu, cuda or {torch.utils.backend_registration._privateuse1_backend_name}"
                )
            elif torch.is_grad_enabled() and any(
                _arg_requires_grad(x) for x in tensor_args
            ):
                why_not_fast_path = (
                    "grad is enabled and at least one of query or the "
                    "input/output projection weights or biases requires_grad"
                )
            if not why_not_fast_path:
                merged_mask, mask_type = self.merge_masks(
                    attn_mask, key_padding_mask, query
                )

                if self.in_proj_bias is not None and self.in_proj_weight is not None:
                    return torch._native_multi_head_attention(
                        query,
                        key,
                        value,
                        self.embed_dim,
                        self.num_heads,
                        self.in_proj_weight,
                        self.in_proj_bias,
                        self.out_proj.weight,
                        self.out_proj.bias,
                        merged_mask,
                        need_weights,
                        average_attn_weights,
                        mask_type,
                    )

        any_nested = query.is_nested or key.is_nested or value.is_nested
        assert not any_nested, (
            "MultiheadAttention does not support NestedTensor outside of its fast path. "
            + f"The fast path was not hit because {why_not_fast_path}"
        )

        if self.batch_first and is_batched:
            # make sure that the transpose op does not affect the "is" property
            if key is value:
                if query is key:
                    query = key = value = query.transpose(1, 0)
                else:
                    query, key = (x.transpose(1, 0) for x in (query, key))
                    value = key
            else:
                query, key, value = (x.transpose(1, 0) for x in (query, key, value))

        if not self._qkv_same_embed_dim:
            attn_output, attn_output_weights = F.multi_head_attention_forward(
                query,
                key,
                value,
                self.embed_dim,
                self.num_heads,
                self.in_proj_weight,
                self.in_proj_bias,
                self.bias_k,
                self.bias_v,
                self.add_zero_attn,
                self.dropout,
                self.out_proj.weight,
                self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask,
                need_weights=need_weights,
                attn_mask=attn_mask,
                use_separate_proj_weight=True,
                q_proj_weight=self.q_proj_weight,
                k_proj_weight=self.k_proj_weight,
                v_proj_weight=self.v_proj_weight,
                average_attn_weights=average_attn_weights,
                is_causal=is_causal,
            )
        else:
            attn_output, attn_output_weights = F.multi_head_attention_forward(
                query,
                key,
                value,
                self.embed_dim,
                self.num_heads,
                self.in_proj_weight,
                self.in_proj_bias,
                self.bias_k,
                self.bias_v,
                self.add_zero_attn,
                self.dropout,
                self.out_proj.weight,
                self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask,
                need_weights=need_weights,
                attn_mask=attn_mask,
                average_attn_weights=average_attn_weights,
                is_causal=is_causal,
            )
        if self.batch_first and is_batched:
            return attn_output.transpose(1, 0), attn_output_weights
        else:
            return attn_output, attn_output_weights

    def merge_masks(
        self,
        attn_mask: Optional[Tensor],
        key_padding_mask: Optional[Tensor],
        query: Tensor,
    ) -> tuple[Optional[Tensor], Optional[int]]:
        r"""Determine mask type and combine masks if necessary.

        If only one mask is provided, that mask
        and the corresponding mask type will be returned. If both masks are provided, they will be both
        expanded to shape ``(batch_size, num_heads, seq_len, seq_len)``, combined with logical ``or``
        and mask type 2 will be returned
        Args:
            attn_mask: attention mask of shape ``(seq_len, seq_len)``, mask type 0
            key_padding_mask: padding mask of shape ``(batch_size, seq_len)``, mask type 1
            query: query embeddings of shape ``(batch_size, seq_len, embed_dim)``
        Returns:
            merged_mask: merged mask
            mask_type: merged mask type (0, 1, or 2)
        """
        mask_type: Optional[int] = None
        merged_mask: Optional[Tensor] = None

        if key_padding_mask is not None:
            mask_type = 1
            merged_mask = key_padding_mask

        if attn_mask is not None:
            # In this branch query can't be a nested tensor, so it has a shape
            batch_size, seq_len, _ = query.shape
            mask_type = 2

            # Always expands attn_mask to 4D
            if attn_mask.dim() == 3:
                attn_mask_expanded = attn_mask.view(batch_size, -1, seq_len, seq_len)
            else:  # attn_mask.dim() == 2:
                attn_mask_expanded = attn_mask.view(1, 1, seq_len, seq_len).expand(
                    batch_size, self.num_heads, -1, -1
                )
            merged_mask = attn_mask_expanded

            if key_padding_mask is not None:
                key_padding_mask_expanded = key_padding_mask.view(
                    batch_size, 1, 1, seq_len
                ).expand(-1, self.num_heads, -1, -1)
                merged_mask = attn_mask_expanded + key_padding_mask_expanded

        # no attn_mask and no key_padding_mask, returns None, None
        return merged_mask, mask_type




In [144]:
attention = nn.MultiheadAttention(40, 2, kdim=10, vdim=20);
attention = CrossMultiheadAttention(10, 2, qdim=10, kdim=10, vdim=10, batch_first=True);
x = torch.randn((10, 14, 10));
y = torch.randn((10, 10, 10));
attention(x, x, y)

AssertionError: key shape torch.Size([14, 10, 10]) does not match value shape torch.Size([10, 10, 10])

In [148]:
_, x = torch.randn((n_seq, batch_size, qdim));
print(x)

ValueError: too many values to unpack (expected 2)

In [147]:
n_seq = 10;
batch_size = 14;
qdim = 10;
kdim = 12;
vdim = 12;
embed_dim = 20;
out_dim = 17;
F.multi_head_attention_forward(
                torch.randn((n_seq, batch_size, qdim)),
                torch.randn((n_seq, batch_size, kdim)),
                torch.randn((n_seq, batch_size, vdim)),
                embed_dim,
                2,
                None,
                None,
                None,
                None,
                False,
                0.0,
                torch.randn((out_dim, embed_dim)),
                None,
                use_separate_proj_weight=True,
                q_proj_weight=torch.randn((qdim, embed_dim)),
                k_proj_weight=torch.randn((embed_dim, kdim)),
                v_proj_weight=torch.randn((embed_dim, vdim)),
            )[0].shape

AssertionError: was expecting embedding dimension of 20, but got 10

# Serious code
## Imports

In [1]:
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F
import torch

import time
import numpy as np
import jax

from gymnax.environments import environment
from simulators.clifford_gates import CliffordGates
from envs.logical_state_preparation_env import LogicalStatePreparationEnv


## Class definitions

In [2]:
class Layout:
    graph : torch.Tensor
    adjacency_list : list[tuple[int, int]]
    adjacency_oh_matrices : tuple[Tensor, Tensor]

    def __init__(self, graph : Tensor) -> None:
        self.graph = graph.int()
        self.adjacency_list = Layout.construct_adjacency_list(self.graph)
        self.adjacency_oh_matrices = Layout.construct_adjacency_oh_matrices(self.graph)

    @staticmethod
    def construct_adjacency_list(graph: torch.Tensor) -> list[tuple[int, int]]:
        adj_list = []
        for q_idx, row in enumerate(graph):
            for t_idx, el in enumerate(row):
                # Assumes `graph` is an int tensor
                if q_idx > t_idx and (el == 1 or el == 1.):
                    adj_list.append((q_idx, t_idx))
                    adj_list.append((t_idx, q_idx))
        return adj_list

    @staticmethod
    def construct_adjacency_oh_matrices(graph: torch.Tensor) -> tuple[Tensor, Tensor]:
        """Construct matrices which one-hot encodings of the adjacency list representation.
        If (i, j) is a directed edge, then the first matrix will have one_hot(i) vector and the
        second will have one_hot(j) vector. The size of the matrices will be #edges * #nodes.
        """
        ctrl_list, tgt_list = [], []
        for q_idx, row in enumerate(graph):
            for t_idx, el in enumerate(row):
                if q_idx > t_idx and (el == 1 or el == 1.):
                    ctrl_list.append(q_idx)
                    tgt_list.append(t_idx)

                    ctrl_list.append(t_idx)
                    tgt_list.append(q_idx)
        nq = graph.shape[-1]
        return F.one_hot(torch.tensor(ctrl_list), num_classes=nq), F.one_hot(torch.tensor(tgt_list), num_classes=nq)
        
def sample_layout(n : int, gen : torch.Generator | None = None) -> Layout:
    """Construct a random graph with n vertices.

    Args:
        n: number of vertices
        rng: Random number generator
    
    Returns:
        Graph laplacian
    """
    satisfactory = False
    while not satisfactory:
        graph = torch.empty(n, n).uniform_(0, 1, generator=gen)
        graph = torch.bernoulli(graph, generator=gen)
        graph = torch.triu(graph, diagonal=1)
        graph = graph + torch.transpose(graph, 0, 1)
        graph = graph - torch.diag(graph.sum(dim=0))
        graph = graph.int()
        satisfactory = torch.all(graph.diag(0) != 0)
    return Layout(graph)

In [3]:
from enum import Enum

class GT_1Q(Enum):
    """Gate Type - 1 Qubit
    """
    H = 1
    S = 2
    X = 3
    SQRT_X = 4

class GT_2Q(Enum):
    """Gate Type - 2 Qubit
    Model parameters, sampling procedure etc. will depend on the number of gates defined in
    this Enum. Any new gates added to the Enum can break the working functionality of a given
    model and set up.
    """
    CX = 1
    CZ = 2
    # SWAP = 3 # Unsupported
    # I_SWAP = 4 # Unsupported

## Torch modules

In [4]:
def _cartesian_add(x: Tensor, y: Tensor) -> Tensor:
    """ Consider x, y to be 2D tensors of dim (nx, dx) and (ny, dy) where dx = dy.
    Then, the result will be a 2D tensor of dim (nx*ny, dx) with the following elements:
    [x[0] + y[0], x[0] + y[1], ..., x[0] + y[ny-1], x[1] + y[0], ..., x[nx-1] + y[ny-1]]
    """
    nx, dx = x.shape[-2:]
    ny, dy = y.shape[-2:]
    assert dx == dy, f"Dimensions of x and y should match. Got {dx} and {dy}"

    x = x.unsqueeze(-2) # Shape: (nx, 1, d)
    x = x.expand(*x.shape[:-3], nx, ny, dx) # Shape: (nx, ny, d)
    x = x.reshape(*x.shape[:-3], -1, dx)

    y = y.unsqueeze(-3) # Shape: (1, ny, d)
    y = y.expand(*y.shape[:-3], nx, ny, dx) # Shape: (nx, ny, d)
    y = y.reshape(*y.shape[:-3], -1, dx)
    return x+y

def _generate_qubit_pair(qubits: Tensor, layout: Layout) -> Tensor:
    """ Given a layout of qubits with specified connectivity, and the qubit embeddings [q[0], ..., q[n-1]]
    return a tensor of concat(q[i], q[j]) where (i, j) forms an edge in the graph layout.
    """
    ctrl, tgt = layout.adjacency_oh_matrices
    return torch.concat((
        torch.matmul(ctrl.float(), qubits),
        torch.matmul(tgt.float(), qubits)
    ), dim=-1)
    

In [15]:
class Tokens:
    def __init__(self, A: Tensor, B: Tensor, C: Tensor, D: Tensor, E: Tensor) -> None:
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        self.E = E

class TokenProperties:
    def __init__(self, truncate_eigval: int, gate_type_1q_embed_dim: int, stabilizer_embed_dim: int, tableau_embed_dim: int):
        self.gate_type_1q_embed_dim = gate_type_1q_embed_dim
        self.stabilizer_embed_dim = stabilizer_embed_dim
        self.tableau_embed_dim = tableau_embed_dim

        self.dA = truncate_eigval
        self.init_token_dims()
    
    def init_token_dims(self):
        self.dB = self.gate_type_1q_embed_dim
        self.dC = self.dB * 2
        self.dD = self.stabilizer_embed_dim
        self.dE = self.tableau_embed_dim

In [16]:
class PositionalEncoding(nn.Module):
    """Positional encodings for transformers. We use sinusoidal positional embeddings, but
    in a form different than usual positional encoding for transformers. Normally, we have a N tokens
    as input (where N is fixed), and we construct N positional vectors while truncating the embedding
    dimension to match the dimension of each token vector. This behavior also means the encodings can
    be constructed at initialization. Instead, here N is not fixed. So we will construct new positional
    encodings for each input.

    Given an input tensor of size (..., m, n) the output will be a tensor of size (m, embedding_dim)

    Also a nice explaination on why sinusoidal encodings:
    https://kazemnejad.com/blog/transformer_architecture_positional_encoding/
    """
    def __init__(self, embedding_dim: int = 50) -> None:
        super().__init__(self)
        self.embedding_dim = embedding_dim

    def apply(self, x: Tensor) -> Tensor:
        # Interpret the last 2 dimensions of x as matrix dimensions and derive the positional
        # encodings based on the shape. In row vector format, (n_rows, n_columns) refers to
        # `n_rows` number of vectors to work with, and `n_columns` is the dimension of each vector.
        n, _d = x.shape[-2:]
        numtr = torch.arange(n).unsqueeze(1) # k
        dentr = torch.exp(torch.arange(0, self.embedding_dim, 2) * (-np.log(10000))).unsqueeze(0) # 1 / 10000 ^ (2i/d)

        output = torch.zeros((n, self.embedding_dim))
        # PE(k, 2i) = sin(k / 10000^(2i / d))
        output[:, 0::2] = torch.sin(numtr * dentr)
        # PE(k, 2i+1) = cos(k / 10000^(2i / d))
        output[:, 1::2] = torch.cos(numtr * dentr)
        return output

class Gate1QEmbedding(nn.Module):
    """Learnt embeddings for each 1-qubit gate type. It implicitly makes use of the properties of
    the Enum GT_1Q, which means the model would need to be retrained if it is tweaked.

    Given an input gate set of size G and n qubits (tensor size (n, q_embedding_dim)) the output will
    be a tensor of size (G*n, embedding_dim)
    """
    def __init__(self, embedding_dim: int = 30) -> None:
        super().__init__()
        # Use nn.Linear for proper initialization.
        self.embedding_dim = embedding_dim
        layer = nn.Linear(len(GT_1Q), embedding_dim, bias=False)
        self.weights = layer(F.one_hot(torch.arange(0, len(GT_1Q))).float())

    def apply(self, gate_set: list[GT_1Q], qubits: Tensor) -> Tensor:
        """Generate embeddings of each pair (Gate, Qubit) where qubit index moves first.
        """
        # (-1) to ensure indexing starts from 0
        gate_idxs = torch.tensor([gate.value for gate in gate_set], dtype=torch.int64) - 1
        gates_oh = F.one_hot(gate_idxs, num_classes=len(GT_1Q)).float()
        gate_embeddings = torch.matmul(gates_oh, self.weights)
        result = _cartesian_add(gate_embeddings, qubits)
        return result

class Gate2QEmbedding(nn.Module):
    """Learnt embeddings for each 2-qubit gate type. It implicitly makes use of the properties of
    the Enum GT_2Q, which means the model would need to be retrained if it is tweaked.

    Given an input gate set of size G, n qubits (tensor size (n, q_embedding_dim)), and a layout with
    E edges, the output will be a tensor of size (E * G, embedding_dim)
    """
    def __init__(self, embedding_dim: int = 60) -> None:
        super().__init__()
        # Use nn.Linear for proper initialization.
        self.embedding_dim = embedding_dim
        layer = nn.Linear(len(GT_2Q), embedding_dim, bias=False)
        self.weights = layer(F.one_hot(torch.arange(0, len(GT_2Q))).float())

    def apply(self, gate_set: list[GT_2Q], qubits: Tensor, layout: Layout) -> Tensor:
        """Generate embeddings of each pair (Gate, Qubit-Qubit) where qubit index moves first.
        """
        # (-1) to ensure indexing starts from 0
        gate_idxs = torch.tensor([gate.value for gate in gate_set], dtype=torch.int64) - 1
        gates_oh = F.one_hot(gate_idxs, num_classes=len(GT_2Q)).float()
        gate_embeddings = torch.matmul(gates_oh, self.weights)
        result = _cartesian_add(gate_embeddings, _generate_qubit_pair(qubits, layout))
        return result

class StabilizerEmbedding(nn.Module):
    """Learnt embeddings for each stabilizer sign.
    
    Given an input tensor of size 2*n*n+n the output will a tensor of size (n, embedding_dim).
    """
    def __init__(self, embedding_dim: int = 60) -> None:
        super().__init__()
        # Use nn.Linear for proper initialization.
        self.embedding_dim = embedding_dim
        self.n_signs = 2
        # input dim = 2 because there are only 2 signs
        layer = nn.Linear(self.n_signs, embedding_dim, bias=False)
        self.weights = layer(F.one_hot(torch.arange(0, self.n_signs)).float())
        self.positional_encoding = PositionalEncoding(embedding_dim)
    
    def apply(self, nq: int, observation: Tensor) -> Tensor:
        """
        Args:
            nq: number of qubits
            observation: output of (bool) check matrix. The last dimension should have the format 
                (X1 X2 ... Xnq Z1 Z2 .. Znq)_1 ... (X1 X2 ... Xnq Z1 Z2 .. Znq)_nq (S_1 ... S_nq)
                where ()_i represents the ith stabilizer, S_i represents the sign of the ith
                stabilizer
        """
        signs = torch.narrow(observation, -1, -nq, nq)
        signs_oh = F.one_hot(signs, num_classes=self.n_signs).float()
        sign_embeddings = torch.matmul(signs_oh, self.weights)
        return sign_embeddings + self.positional_encoding(sign_embeddings)

class TableauCellEmbedding(nn.Module):
    """I, X, Z, Y
    """
    def __init__(self, embedding_dim: int = 60) -> None:
        super().__init__()
        # Use nn.Linear for proper initialization.
        self.embedding_dim = embedding_dim
        self.n_paulis = 4
        # input dim = 4 because there are only 4 Paulis
        layer = nn.Linear(self.n_paulis, embedding_dim, bias=False)
        self.weights = layer(F.one_hot(torch.arange(0, self.n_paulis)).float())
        self.positional_encoding = PositionalEncoding(embedding_dim)
    
    def apply(self, nq: int, observation: Tensor, qubits: Tensor) -> Tensor:
        # Map the Paulis as follow: 0 -> I, 1 -> X, 2 -> Y, 3 -> Z
        paulis = torch.narrow(observation, -1, 0, nq * nq) + 2 * torch.narrow(observation, -1, nq * nq, 2 * nq * nq)
        # paulis = paulis[..., 0:nq * nq] + paulis[..., nq * nq:]
        paulis_oh = F.one_hot(paulis, num_classes=self.n_paulis)
        pauli_embeddings = torch.matmul(paulis_oh, self.weights)
        qubits = qubits.unsqueeze(-3) # Shape: (1, nq, d)
        qubits = qubits.expand(*qubits.shape[:-3], nq, nq, -1) # Shape: (nq, nq, d)
        return qubits + pauli_embeddings

class ProjectionLayer(nn.Module):
    def apply(self, x: Tensor) -> Tensor:
        # Interpret the last 2 dimensions of x as a matrix and derive the positional encodings
        # based on the shape
        raise NotImplementedError("Model not yet implemented")

class ResidualLayer(nn.Module):
    def __init__(self, alpha: float=0.80) -> None:
        self.alpha = alpha
    
    def apply(self, x: Tensor | Tokens, f_x: Tensor | Tokens) -> Tensor | Tokens:
        assert type(x) == type(f_x), "Either both inputs should be Tensors or Tokens"
        if isinstance(x, Tokens) and isinstance(f_x, Tokens):
            return Tokens(
                x.A + self.alpha * f_x.A,
                x.B + self.alpha * f_x.B,
                x.C + self.alpha * f_x.C,
                x.D + self.alpha * f_x.D,
                x.E + self.alpha * f_x.E
            )
        else:
            return x + self.alpha * f_x
        
class HomogenousAttentionBlock(nn.Module):
    def __init__(self, token_dim: TokenProperties, n_head: int=2) -> None:
        super().__init__()
        self.layer_B = nn.TransformerEncoderLayer(token_dim.dB, n_head)
        self.layer_C = nn.TransformerEncoderLayer(token_dim.dC, n_head)
        self.layer_D = nn.TransformerEncoderLayer(token_dim.dD, n_head)
        self.layer_E = nn.TransformerEncoderLayer(token_dim.dE, n_head)

    def apply(self, x: Tokens) -> Tokens:
        return Tokens(
            x.A,
            self.layer_B(x.B),
            self.layer_C(x.C),
            self.layer_D(x.D),
            self.layer_E(x.E)
        )

class HeterogenousAttentionBlock(nn.Module):
    def __init__(self, token_dims: TokenProperties, embedding_dim: int=50, n_head: int=4) -> None:
        super().__init__()
        self.token_dims = token_dims
        self.embedding_dim = embedding_dim
        self.n_head = n_head

        self.softmax = nn.Softmax(-1)
        self.residual_layer = ResidualLayer()
        self.init_attention_layers()
        self.init_feedforward_networks()
    
    def init_attention_layers(self) -> None:
        self.kwA = nn.Linear(self.token_dims.dA, self.embedding_dim)
        self.kwB = nn.Linear(self.token_dims.dB, self.embedding_dim)
        self.kwC = nn.Linear(self.token_dims.dC, self.embedding_dim)
        self.kwD = nn.Linear(self.token_dims.dD, self.embedding_dim)
        self.kwE = nn.Linear(self.token_dims.dE, self.embedding_dim)

        self.qwA = nn.Linear(self.token_dims.dA, self.embedding_dim)
        self.qwB = nn.Linear(self.token_dims.dB, self.embedding_dim)
        self.qwC = nn.Linear(self.token_dims.dC, self.embedding_dim)
        self.qwD = nn.Linear(self.token_dims.dD, self.embedding_dim)
        self.qwE = nn.Linear(self.token_dims.dE, self.embedding_dim)

        self.vwA = nn.Linear(self.token_dims.dA, self.embedding_dim)
        self.vwB = nn.Linear(self.token_dims.dB, self.embedding_dim)
        self.vwC = nn.Linear(self.token_dims.dC, self.embedding_dim)
        self.vwD = nn.Linear(self.token_dims.dD, self.embedding_dim)
        self.vwE = nn.Linear(self.token_dims.dE, self.embedding_dim)

    def init_feedforward_networks(self) -> None:
        self.fcn_B = nn.Linear(self.embedding_dim * 4, self.token_dims.dB)
        self.fcn_C = nn.Linear(self.embedding_dim * 4, self.token_dims.dC)
        self.fcn_D = nn.Linear(self.embedding_dim * 4, self.token_dims.dD)
        self.fcn_E = nn.Linear(self.embedding_dim * 4, self.token_dims.dE)

    def get_attention_output(self, q: Tensor, kv_pairs: list[tuple[Tensor, Tensor]]) -> Tensor:
        def single_attention_output(q: Tensor, k: Tensor, v: Tensor) -> Tensor:
            q = torch.unsqueeze(q, -1).reshape(*q.shape[:-2], self.n_head, -1)
            k = torch.unsqueeze(k, -1).reshape(*k.shape[:-2], self.n_head, -1)
            v = torch.unsqueeze(v, -1).reshape(*v.shape[:-2], self.n_head, -1)
            weights = self.softmax(torch.matmul(q, k.transpose(-2,-1)) / torch.sqrt(torch.Tensor(self.embedding_dim)))
            output = torch.matmul(weights, v)
            output = output.reshape(*output.shape[:-2], -1, 1)
            output = torch.squeeze(output, -1)
            return output
        
        return torch.concat([single_attention_output(q, k, v) for k, v in kv_pairs], dim=-1)

    def apply(self, x: Tokens) -> Tokens:
        kA, qA, vA = self.kwA(x.A), self.qwA(x.A), self.vwA(x.A)
        kB, qB, vB = self.kwB(x.B), self.qwB(x.B), self.vwB(x.B)
        kC, qC, vC = self.kwC(x.C), self.qwC(x.C), self.vwC(x.C)
        kD, qD, vD = self.kwD(x.D), self.qwD(x.D), self.vwD(x.D)
        kE, qE, vE = self.kwE(x.E), self.qwE(x.E), self.vwE(x.E)

        attention_to_B = self.get_attention_output(qB, [(kC, vC), (kD, vD), (kE, vE), (kA, vA)])
        attention_to_C = self.get_attention_output(qC, [(kD, vD), (kE, vE), (kA, vA), (kB, vB)])
        attention_to_D = self.get_attention_output(qD, [(kE, vE), (kA, vA), (kB, vB), (kC, vC)])
        attention_to_E = self.get_attention_output(qE, [(kA, vA), (kB, vB), (kC, vC), (kD, vD)])

        additive_inp_B = self.fcn_B(attention_to_B)
        additive_inp_C = self.fcn_C(attention_to_C)
        additive_inp_D = self.fcn_D(attention_to_D)
        additive_inp_E = self.fcn_E(attention_to_E)

        return Tokens(
            x.A, 
            self.residual_layer(x.B, additive_inp_B),
            self.residual_layer(x.C, additive_inp_C),
            self.residual_layer(x.D, additive_inp_D),
            self.residual_layer(x.E, additive_inp_E)
        )

In [6]:
from gymnax.environments import EnvState

class ModelV0(nn.Module):
    def __init__(
        self,
        nq: int,
        num_transformer_blocks: int = 2, 
        positional_encoding_dim: int = 50, 
        embedding_dims: TokenProperties | None = None
    ) -> None:
        # TODO: initialize `embedding_dims` properly
        super().__init__()
        self.num_transformer_blocks = num_transformer_blocks
        self.positional_encoding = PositionalEncoding(positional_encoding_dim)
        self.GT_1Q_embedding = Gate1QEmbedding(embedding_dims.gate_type_1q_embed_dim)
        self.GT_2Q_embedding = Gate2QEmbedding(2 * embedding_dims.gate_type_1q_embed_dim)
        self.stabilizer_embedding = StabilizerEmbedding(embedding_dims.stabilizer_embed_dim)
        self.tableau_cell_embedding = TableauCellEmbedding(embedding_dims.tableau_embed_dim)
        self.heterogenous_attention_block = nn.ModuleList(
            [HeterogenousAttentionBlock(embedding_dims) for _ in range(self.num_transformer_blocks)])
        self.computation_layer = nn.ModuleList(
            [HomogenousAttentionBlock(embedding_dims) for _ in range(self.num_transformer_blocks)])
        self.projection_layer = ProjectionLayer()
        self.residual_layer = ResidualLayer()

    def apply(self, layout: Layout, gate_set_1q: list[GT_1Q], gate_set_2q: list[GT_2Q], observation: Tensor) -> Tensor:
        # Token A -> Global token representing the graph
        # Note:
        # 1. `graph_eigvec` -> each column represents an eigenvector
        # 2. eigh is specifically for Hermitian matrices. If the input is a real matrix,
        #    the eigenvectors and eigenvalues are both guaranteed to be reals as well.
        graph_eigval, graph_eigvec = torch.linalg.eigh(layout.graph)
        # Token B
        # Q: Is there no learned embedding for the qubits? It will be used in stabilizer and
        # tableau-cell tokens as well. There seems to be a very small amount of learnable
        # components then.
        qubit_tensors = torch.cat(graph_eigvec, self.positional_encoding(graph_eigvec))
        # Token C
        gate_tensors = torch.stack(
            self.GT_1Q_embedding(gate_set_1q, qubit_tensors),
            self.GT_2Q_embedding(gate_set_2q, qubit_tensors, layout)
        )
        # Token D
        stabilizer_tensors = self.stabilizer_embedding(observation)
        # Token E
        tableau_cell_tensors = self.tableau_cell_embedding(observation, qubit_tensors)

        x = Tokens(graph_eigval, qubit_tensors, gate_tensors, stabilizer_tensors, tableau_cell_tensors);
        for i in range(self.num_transformer_blocks):
            attention_output = self.heterogenous_attention_block[i](x)
            x = self.residual_layer(x, attention_output)
            computation_layer = self.computation_layer[i](x)
            x = self.residual_layer(x, computation_layer)
        return self.projection_layer(x)

## Interacting with the environment

In [7]:
from typing import Any

def sample_gate_set(gen : torch.Generator | None = None) -> tuple[list[GT_1Q], list[GT_2Q]]:
    """Construct a random graph with n vertices.

    Args:
        n: number of vertices
        rng: Random number generator
    
    Returns:
        Graph laplacian
    """
    gate_set_1q = [gate for gate in GT_1Q if torch.rand(1, generator=gen).item() < 0.5]
    gate_set_2q = [gate for gate in GT_2Q if torch.rand(1, generator=gen).item() < 0.5]
    return (gate_set_1q, gate_set_2q)

def construct_gate_instances(gate_set_1q : list[GT_1Q], gate_set_2q : list[GT_2Q], graph : torch.Tensor) -> list[Any]:
    gate_list = []
    for q_idx, row in enumerate(graph):
        gate_list = gate_list + [(gate, q_idx) for gate in gate_set_1q]
        for t_idx, el in enumerate(row):
            if torch.eq(el, torch.Tensor(1)):
                for gate in gate_set_2q:
                    if gate == GT_2Q.CZ:
                        # Control and target are interchangeable
                        if q_idx > t_idx:
                            gate_list = gate_list + [(gate, q_idx, t_idx) for gate in gate_set_2q]
                    else:
                        gate_list = gate_list + [(gate, q_idx, t_idx) for gate in gate_set_2q]
    return gate_list

def sample_from_array(inp_list, d, gen : torch.Generator | None = None):
    # Change the input from the gate instance list (size `G`) which contains Enums to a Tensor which contains
    # G tensors representing embeddings of the different gate instances. This change will also make indexing
    # easier.
    weights = torch.ones(len(inp_list))
    indices = torch.multinomial(weights, d, generator=gen, replacement=True)
    return [inp_list[idx] for idx in indices];
    # return inp_list[indices];

def init_env(layout: Layout, gate_set_1q : list[GT_1Q], gate_set_2q : list[GT_2Q], max_steps: int) -> LogicalStatePreparationEnv:
    n = layout.graph.shape[0]
    # Set target to be the all 0 state.
    identity_string = f"{"".join(["I" for _ in range(n)])}"
    target = ["+" + identity_string[:i] + "Z" + identity_string[i+1:] for i in range(n)]

    # Set the gates to be the ones we sampled.
    clifford_gates = CliffordGates(n);
    gate_list = []
    for gate in gate_set_1q:
        match gate:
            case GT_1Q.H:
                gate_list.append(clifford_gates.h)
            case GT_1Q.S:
                gate_list.append(clifford_gates.s)
            case GT_1Q.X:
                gate_list.append(clifford_gates.x)
            case GT_1Q.SQRT_X:
                gate_list.append(clifford_gates.sqrt_x)
    for gate in gate_set_2q:
        match gate:
            case GT_2Q.CX:
                gate_list.append(clifford_gates.cx)
            case GT_2Q.CZ:
                gate_list.append(clifford_gates.cz)
            # case GT_2Q.SWAP:
            #     raise NotImplementedError("SWAP gate not supported in LSP environment yet")
            # case GT_2Q.I_SWAP:
            #     raise NotImplementedError("iSWAP gate not supported in LSP environment yet")

    # Prep the graph input.
    return LogicalStatePreparationEnv(
        target,
        gates = gate_list,
        graph = layout.adjacency_list,
        distance_metric = 'jaccard',
        max_steps = max_steps,
        threshold = 0.99,
        use_max_reward = False
    )


In [8]:
import numpy as np
import time

seed = time.time_ns();

# JAX RNG
key = jax.random.key(seed)
key, key_reset, key_act, key_step = jax.random.split(key, 4)

# Torch RNG
gen = torch.Generator()
gen.manual_seed(seed)

n_min, n_max = 1, 20

sl_model = ModelV0();

while loss is bad enough:
    n = torch.randint(low=n_min, high=n_max, size=(1,), generator=gen)[0]
    n = n.int().item()

    layout = Layout.sample_layout(n, gen)
    gate_set_1q, gate_set_2q = sample_gate_set(gen)
    # gate_instances = construct_gate_instances(gate_set_1q, gate_set_2q, graph)

    d_max =  np.floor(n * n / np.log2(len(gate_set_1q) + len(gate_set_2q)), casting='unsafe', dtype=np.int32)
    d = torch.randint(low=1, high=d_max, size=(1,), generator=gen)[0]
    # gates = sample_from_array(gate_instances, d, gen)

    lsp_env = init_env(layout, gate_set_1q, gate_set_2q, d)
    env_params = None
    _observation, env_state = lsp_env.reset_env(key_reset, env_params);
    assert _observation.shape[-1] == 2 * n * n + n, f"""Implementation (e.g. StabilizerEncoding) depends on the shape
    of the observation being 2 * n * n + n (all stabilizers, followed by n signs). Received {_observation.shape}
    instead"""

    gate_list = []
    for idx in range(d):
        key_act, _rng = jax.random.split(key_act);
        gate_list.append(lsp_env.action_space(env_params).sample(key_act))

        key_step, _rng = jax.random.split(key_step);
        _observation, env_state, _reward, _done, _info = lsp_env.step_env(key_step, env_state, gate_list[-1], env_params)
    
    gate_prediction = sl_model.apply(layout, gate_set_1q, gate_set_2q, env_state)
    loss = loss_function(gate_prediction, gate_list[-1])
    network.backprop(loss)

SyntaxError: invalid syntax (2619343340.py, line 18)

Observation vs state?

In [9]:
import numpy as np
import time

seed = time.time_ns();
gen = None

# JAX RNG
key = jax.random.key(seed)
key, key_reset, key_act, key_step = jax.random.split(key, 4)

n = torch.randint(low=1, high=5, size=(1,), generator=gen)[0]
n = n.int().item()

layout = sample_layout(n, gen)
gate_set_1q, gate_set_2q = sample_gate_set(gen)
# gate_instances = construct_gate_instances(gate_set_1q, gate_set_2q, graph)

d_max =  np.floor(n * n / np.log2(len(gate_set_1q) + len(gate_set_2q)), casting='unsafe', dtype=np.int32)
d = torch.randint(low=1, high=d_max, size=(1,), generator=gen)[0]
# gates = sample_from_array(gate_instances, d, gen)

lsp_env = init_env(layout, gate_set_1q, gate_set_2q, d)
env_params = None
_observation, env_state = lsp_env.reset_env(key_reset, env_params);

print(_observation)

/Users/tport/miniconda3/envs/RL-LSP/lib/python3.13/site-packages/jax/_src/ops/scatter.py:108: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=int32 to dtype=uint8 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


[0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]


In [14]:
print(_observation.shape[-1])

11


In [46]:
western_notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
hindustani_notes = ["S", "R_", "R", "G_", "G", "M", "M+", "P", "D_", "D", "N_", "N"]
def transpose(
    input_notes: list[str], 
    first_note: str|None=None, 
    shift: int|None=None, 
    transpose_from: str|None=None, 
    transpose_to: str|None=None
    ) -> list[str]:
    if first_note is not None:
        shift = western_notes.index(first_note) - western_notes.index(input_notes[0])
        if shift < 0:
            shift = shift + 12
    if transpose_from is not None and transpose_to is not None:
        shift = western_notes.index(transpose_to) - western_notes.index(transpose_from)
        if shift < 0:
            shift = shift + 12
    assert shift is not None, "Arguments could not be identified for calculating shift"
    input_idx = [western_notes.index(note) for note in input_notes]
    shifted_idx = [(idx + shift) % 12 for idx in input_idx]
    return [western_notes[idx] for idx in shifted_idx]

def convert_to_hindustani(input_notes: list[str], base_note: str) -> list[str]:
    shift = western_notes.index(base_note)
    input_idx = [western_notes.index(note) for note in input_notes]
    shifted_idx = [(idx - shift + 12) % 12 for idx in input_idx]
    return [hindustani_notes[idx] for idx in shifted_idx]
    


In [48]:
# input_notes = ["B", "G", "G", "A"] # Bachana
input_notes = ["D", "C", "D", "E", "F", "G", "G", "F", "E"] # Bachana - lyrics
# input_notes = ["D", "E", "A", "B", "F#", "E", "F#", "E", "C#"]
notes = input_notes

print("Western:")
for i in western_notes:
    print(f"{i}\t{transpose(notes, first_note=i)}")

print()
print("Hindustani:")
# notes = transpose(notes, transpose_from="C", transpose_to="E")
print(notes)
for i in western_notes:
    print(f"{i}\t{convert_to_hindustani(notes, base_note=i)}")

Western:
C	['C', 'A#', 'C', 'D', 'D#', 'F', 'F', 'D#', 'D']
C#	['C#', 'B', 'C#', 'D#', 'E', 'F#', 'F#', 'E', 'D#']
D	['D', 'C', 'D', 'E', 'F', 'G', 'G', 'F', 'E']
D#	['D#', 'C#', 'D#', 'F', 'F#', 'G#', 'G#', 'F#', 'F']
E	['E', 'D', 'E', 'F#', 'G', 'A', 'A', 'G', 'F#']
F	['F', 'D#', 'F', 'G', 'G#', 'A#', 'A#', 'G#', 'G']
F#	['F#', 'E', 'F#', 'G#', 'A', 'B', 'B', 'A', 'G#']
G	['G', 'F', 'G', 'A', 'A#', 'C', 'C', 'A#', 'A']
G#	['G#', 'F#', 'G#', 'A#', 'B', 'C#', 'C#', 'B', 'A#']
A	['A', 'G', 'A', 'B', 'C', 'D', 'D', 'C', 'B']
A#	['A#', 'G#', 'A#', 'C', 'C#', 'D#', 'D#', 'C#', 'C']
B	['B', 'A', 'B', 'C#', 'D', 'E', 'E', 'D', 'C#']

Hindustani:
['D', 'C', 'D', 'E', 'F', 'G', 'G', 'F', 'E']
C	['R', 'S', 'R', 'G', 'M', 'P', 'P', 'M', 'G']
C#	['R_', 'N', 'R_', 'G_', 'G', 'M+', 'M+', 'G', 'G_']
D	['S', 'N_', 'S', 'R', 'G_', 'M', 'M', 'G_', 'R']
D#	['N', 'D', 'N', 'R_', 'R', 'G', 'G', 'R', 'R_']
E	['N_', 'D_', 'N_', 'S', 'R_', 'G_', 'G_', 'R_', 'S']
F	['D', 'P', 'D', 'N', 'S', 'R', 'R', 'S', 'N'